# Test for Recommender System using SciKit-Surprise

### import libraries

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
import time
%load_ext autoreload
%autoreload 2

In [2]:
# import scikit-surprise stuff
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate, GridSearchCV

In [3]:
import pandas as pd

In [4]:
# import costum stuff
from utility import *

### Import train and test set

In [5]:
# prepare train set file in correct format for scikit-surprise
filepathcsv = "../data/data_train.csv"
filepath = "../data/data_train.dat"
convertCSV2Surprise(filepathcsv,filepath)

In [6]:
# import train data
csvreader = Reader(line_format=u"user item rating",sep=";",rating_scale=(1,5),skip_lines=1)
data = Dataset.load_from_file(filepath,csvreader)

In [7]:
# prepare test set file in correct format for scikit-surprise
filepathcsv = "../data/sample_submission.csv"
filepath = "../data/sample_submission.dat"
convertCSV2Surprise(filepathcsv,filepath)

In [8]:
# import test data
testdata = Dataset.load_from_file(filepath,csvreader)

In [9]:
# chose algorithm
algo = SVD()

In [10]:
# test sequential execution
t = time.clock()
cross_validate(algo,data,measures=['RMSE','MAE'], cv=2, verbose=True)
t = time.clock()-t
print("Sequential execution time: ",t)

Evaluating RMSE, MAE of algorithm SVD on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    1.0355  1.0368  1.0361  0.0006  
MAE (testset)     0.8295  0.8294  0.8295  0.0000  
Fit time          40.07   40.15   40.11   0.04    
Test time         7.65    7.49    7.57    0.08    
Sequential execution time:  101.35292600000001


In [11]:
# testing parallel execution
t = time.clock()
cross_validate(algo,data,measures=['RMSE','MAE'], cv=2, verbose=True,n_jobs=-1)
t = time.clock()-t
print("Parallel execution time: ",t)

Evaluating RMSE, MAE of algorithm SVD on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    1.0363  1.0355  1.0359  0.0004  
MAE (testset)     0.8296  0.8296  0.8296  0.0000  
Fit time          43.58   38.11   40.84   2.74    
Test time         6.37    5.58    5.97    0.40    
Parallel execution time:  69.39679499999998


In [13]:
grid = \
{\
"n_factors": [i for i in range(10,21,10)],\
#"lr_all": [0.01,0.02,0.005,0.01,0.015,0.02,0.05,0.1],\
"reg_all": [0.04,0.05],\
#"n_epochs": [5,10,20,30,40,50],\
#"biased": [True,False]\
}

gridsearch = GridSearchCV(algo_class=SVD,param_grid=grid,measures=['RMSE','MAE'], cv=5, n_jobs=-1,refit=True,joblib_verbose=2)

In [2]:
[i for i in range(2,25,2)]

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]

In [14]:
gridsearch.fit(data)
print(gridsearch.best_score['rmse'])
print(gridsearch.best_params['rmse'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  9.3min finished


0.9999258721552188
{'n_factors': 20, 'reg_all': 0.04}


In [15]:
results_cv = pd.DataFrame.from_dict(gridsearch.cv_results)

In [16]:
results_cv.to_csv(r"../data/SVD-cv-nfact-regall-2.csv")

In [17]:
full_train = data.build_full_trainset()
best_algo = gridsearch.best_estimator["rmse"]
best_algo.fit(full_train)

In [20]:
predictions = loadData2df("../data/sample_submission.csv")
generatePredictions(best_algo,predictions,2)
exportPredictions("../data/submission_bestOfSVDC-V2.csv",predictions)

user: 37         item: 1          r_ui = None   est = 3.25   {'was_impossible': False}
0 37 1 3.2498740046083197
user: 73         item: 1          r_ui = None   est = 3.07   {'was_impossible': False}
1 73 1 3.0652089190432847
user: 156        item: 1          r_ui = None   est = 3.73   {'was_impossible': False}
2 156 1 3.7306739992361377
user: 160        item: 1          r_ui = None   est = 3.24   {'was_impossible': False}
3 160 1 3.244115804056706
user: 248        item: 1          r_ui = None   est = 3.44   {'was_impossible': False}
4 248 1 3.443722348023373
user: 256        item: 1          r_ui = None   est = 3.41   {'was_impossible': False}
5 256 1 3.410104021187394
user: 284        item: 1          r_ui = None   est = 3.19   {'was_impossible': False}
6 284 1 3.185683855737586
user: 400        item: 1          r_ui = None   est = 3.32   {'was_impossible': False}
7 400 1 3.320381211034498
user: 416        item: 1          r_ui = None   est = 3.55   {'was_impossible': False}
8 416 1 

270 8380 1 3.4029712341474685
user: 8441       item: 1          r_ui = None   est = 2.73   {'was_impossible': False}
271 8441 1 2.7294244878938776
user: 8461       item: 1          r_ui = None   est = 3.10   {'was_impossible': False}
272 8461 1 3.1002389515765767
user: 8468       item: 1          r_ui = None   est = 3.55   {'was_impossible': False}
273 8468 1 3.550392982534696
user: 8478       item: 1          r_ui = None   est = 3.04   {'was_impossible': False}
274 8478 1 3.0355834304331344
user: 8508       item: 1          r_ui = None   est = 3.51   {'was_impossible': False}
275 8508 1 3.507975802016585
user: 8557       item: 1          r_ui = None   est = 3.41   {'was_impossible': False}
276 8557 1 3.4143648558304407
user: 8575       item: 1          r_ui = None   est = 3.50   {'was_impossible': False}
277 8575 1 3.5027886938114334
user: 8576       item: 1          r_ui = None   est = 3.48   {'was_impossible': False}
278 8576 1 3.479626578305965
user: 8611       item: 1          r_u

470 3294 2 3.5512838771511865
user: 3318       item: 2          r_ui = None   est = 3.62   {'was_impossible': False}
471 3318 2 3.616115792105463
user: 3325       item: 2          r_ui = None   est = 3.35   {'was_impossible': False}
472 3325 2 3.3479726362662587
user: 3328       item: 2          r_ui = None   est = 3.84   {'was_impossible': False}
473 3328 2 3.8426391941587363
user: 3350       item: 2          r_ui = None   est = 3.63   {'was_impossible': False}
474 3350 2 3.6341058585209662
user: 3369       item: 2          r_ui = None   est = 3.89   {'was_impossible': False}
475 3369 2 3.889161799691669
user: 3370       item: 2          r_ui = None   est = 3.53   {'was_impossible': False}
476 3370 2 3.5253367587235567
user: 3428       item: 2          r_ui = None   est = 3.43   {'was_impossible': False}
477 3428 2 3.432968223710117
user: 3439       item: 2          r_ui = None   est = 3.12   {'was_impossible': False}
478 3439 2 3.116190747594275
user: 3455       item: 2          r_ui

686 7459 2 3.6476738108725253
user: 7465       item: 2          r_ui = None   est = 3.63   {'was_impossible': False}
687 7465 2 3.6319253099060824
user: 7483       item: 2          r_ui = None   est = 3.44   {'was_impossible': False}
688 7483 2 3.4396849181824694
user: 7504       item: 2          r_ui = None   est = 3.33   {'was_impossible': False}
689 7504 2 3.3343672212208926
user: 7513       item: 2          r_ui = None   est = 3.84   {'was_impossible': False}
690 7513 2 3.8397304719546845
user: 7529       item: 2          r_ui = None   est = 3.56   {'was_impossible': False}
691 7529 2 3.5609205713261396
user: 7537       item: 2          r_ui = None   est = 3.19   {'was_impossible': False}
692 7537 2 3.193445673229224
user: 7545       item: 2          r_ui = None   est = 3.74   {'was_impossible': False}
693 7545 2 3.7356979099862535
user: 7598       item: 2          r_ui = None   est = 3.34   {'was_impossible': False}
694 7598 2 3.342014039426662
user: 7607       item: 2          r_

user: 37         item: 3          r_ui = None   est = 3.55   {'was_impossible': False}
838 37 3 3.5476369995912713
user: 40         item: 3          r_ui = None   est = 3.33   {'was_impossible': False}
839 40 3 3.3326752200065353
user: 43         item: 3          r_ui = None   est = 4.08   {'was_impossible': False}
840 43 3 4.075432330989708
user: 44         item: 3          r_ui = None   est = 3.91   {'was_impossible': False}
841 44 3 3.9095901842869103
user: 63         item: 3          r_ui = None   est = 3.78   {'was_impossible': False}
842 63 3 3.781075561081085
user: 72         item: 3          r_ui = None   est = 3.64   {'was_impossible': False}
843 72 3 3.6356467747523142
user: 73         item: 3          r_ui = None   est = 3.29   {'was_impossible': False}
844 73 3 3.2857322758823275
user: 90         item: 3          r_ui = None   est = 3.53   {'was_impossible': False}
845 90 3 3.5344040000845505
user: 97         item: 3          r_ui = None   est = 3.97   {'was_impossible': Fa

...

22062 4271 14 4.386333297753677
user: 4272       item: 14         r_ui = None   est = 4.15   {'was_impossible': False}
22063 4272 14 4.153137754515391
user: 4274       item: 14         r_ui = None   est = 4.31   {'was_impossible': False}
22064 4274 14 4.3086344840304935
user: 4276       item: 14         r_ui = None   est = 4.40   {'was_impossible': False}
22065 4276 14 4.397171441273457
user: 4280       item: 14         r_ui = None   est = 4.74   {'was_impossible': False}
22066 4280 14 4.7427761421071555
user: 4281       item: 14         r_ui = None   est = 4.55   {'was_impossible': False}
22067 4281 14 4.55408961047296
user: 4283       item: 14         r_ui = None   est = 4.44   {'was_impossible': False}
22068 4283 14 4.441545518402355
user: 4286       item: 14         r_ui = None   est = 4.63   {'was_impossible': False}
22069 4286 14 4.634403100503189
user: 4287       item: 14         r_ui = None   est = 3.69   {'was_impossible': False}
22070 4287 14 3.692036276689785
user: 4288     

user: 5042       item: 14         r_ui = None   est = 4.53   {'was_impossible': False}
22362 5042 14 4.525583933280883
user: 5051       item: 14         r_ui = None   est = 4.47   {'was_impossible': False}
22363 5051 14 4.465262134533737
user: 5058       item: 14         r_ui = None   est = 3.95   {'was_impossible': False}
22364 5058 14 3.9523136265765784
user: 5063       item: 14         r_ui = None   est = 4.51   {'was_impossible': False}
22365 5063 14 4.510617076459396
user: 5070       item: 14         r_ui = None   est = 3.76   {'was_impossible': False}
22366 5070 14 3.756281215812522
user: 5075       item: 14         r_ui = None   est = 4.20   {'was_impossible': False}
22367 5075 14 4.197865228445031
user: 5078       item: 14         r_ui = None   est = 4.07   {'was_impossible': False}
22368 5078 14 4.065885517063041
user: 5080       item: 14         r_ui = None   est = 3.55   {'was_impossible': False}
22369 5080 14 3.54638461616369
user: 5082       item: 14         r_ui = None   

user: 5530       item: 14         r_ui = None   est = 4.35   {'was_impossible': False}
22564 5530 14 4.351742065350189
user: 5532       item: 14         r_ui = None   est = 4.65   {'was_impossible': False}
22565 5532 14 4.648420341881789
user: 5536       item: 14         r_ui = None   est = 3.87   {'was_impossible': False}
22566 5536 14 3.866830337504585
user: 5539       item: 14         r_ui = None   est = 4.07   {'was_impossible': False}
22567 5539 14 4.065525450432626
user: 5548       item: 14         r_ui = None   est = 4.51   {'was_impossible': False}
22568 5548 14 4.510549259076155
user: 5549       item: 14         r_ui = None   est = 4.37   {'was_impossible': False}
22569 5549 14 4.373291327049059
user: 5553       item: 14         r_ui = None   est = 3.83   {'was_impossible': False}
22570 5553 14 3.8266253891534383
user: 5559       item: 14         r_ui = None   est = 4.41   {'was_impossible': False}
22571 5559 14 4.4130105826767565
user: 5560       item: 14         r_ui = None 

user: 5907       item: 14         r_ui = None   est = 3.91   {'was_impossible': False}
22721 5907 14 3.9124413347735256
user: 5908       item: 14         r_ui = None   est = 4.48   {'was_impossible': False}
22722 5908 14 4.475209943156704
user: 5909       item: 14         r_ui = None   est = 4.42   {'was_impossible': False}
22723 5909 14 4.4207692982096995
user: 5911       item: 14         r_ui = None   est = 4.26   {'was_impossible': False}
22724 5911 14 4.264486051358749
user: 5912       item: 14         r_ui = None   est = 4.33   {'was_impossible': False}
22725 5912 14 4.331776032694
user: 5913       item: 14         r_ui = None   est = 3.95   {'was_impossible': False}
22726 5913 14 3.949954731355748


KeyboardInterrupt: 

TODO:

* ~~GridSearchCV~~

* build_full_trainset + algo.predict() -> kaggle

* test other algorithms

* ~~code export to .csv~~

* write intro (Netflix prize, Simon Fuchs, other applications,scikit-surpise (why reinvent the wheel?), short summary of content)

* important: in report show that we did understand the different methods!

* compare speed sgd, 

In [22]:
predictions

,userId,itemId,rating
0,37,1,3.249874
1,73,1,3.065209
2,156,1,3.730674
3,160,1,3.244116
4,248,1,3.443722
5,256,1,3.410104
6,284,1,3.185684
7,400,1,3.320381
8,416,1,3.551488
9,456,1,3.275770


In [26]:
a = "1"

In [27]:
str(a)

'1'

In [28]:
type(algo.trainset.to_raw_iid(0))

str